In [3]:
from urllib.request import urlopen
import os
import requests
from pathlib import Path
import time
import environ

In [57]:
# import webbrowser
# def Driver():
#     try:
#         path = chromedriver_autoinstaller.install()
        
#     except FileNotFoundError as err:
#         print("크롬 브라우저를 찾을 수 없습니다. 설치 후 재시도 하시기 바랍니다.")
#         webbrowser.open("https://www.google.com/intl/ko/chrome/")
#     options = webdriver.ChromeOptions()
#     options.add_experimental_option("excludeSwitches", ["enable-logging"])
#     #options.add_argument("headless")
#     options.add_argument('--start-fullscreen')
#     options.add_argument('--no-sandbox')
#     options.add_argument('--disable-dev-shm-usage')
#     wd = webdriver.Chrome(options = options)
#     return wd

selenium과 bs4로 element로 이미지의 url를 추출하려했으나, 웹사이트 보안 정책인지 뭔지는 몰라도 element를 찾을 수가 없어서 추출이 불가 했다.
그런 이유로 unplash에서 권장하는 정책으로 Rest API 요쳥을 통해서 image URL을 추출했다. 더욱 빠르기도하고 방법도 간단했다.
다만 demo버전의 권한으로는 시간당 50번의 요청밖에 하지 못하고, 페이지당 최대 30개의 이미지를 추출할 수 있어서 최대 50 x 30 시간 당 최대 1500개 까지만 크롤링이 가능하다는 단점이 있다.

In [83]:
class crawler:
    # 환경 변수 세팅
    #BASE_DIR = Path(__file__).resolve().parent
     
    def __init__(self, query, start_page=1):
        BASE_DIR = os.getcwd()
        env = environ.Env(DEBUG=(bool, True))
        environ.Env.read_env(env_file=os.path.join(BASE_DIR, '.env'))  
        self.query = query
        self.start_page = start_page
        self.images = []
        self.is_activate = False
        self.env = env

    def activate(self):
        """
        필요한 검색어에 따라 모든 사진을 크롤링할 때까지 작동시킵니다.
        """
        i=1
        res = requests.get(f'https://api.unsplash.com/search/photos/?client_id={self.env("ACCESS_KEY")}&query={self.query}&page={i}&per_page=30')
        response = res.json()
        total_page = response['total_pages']
        print(f"total_pages = {total_page}")
        while self.start_page <= total_page:
            for i in range(self.start_page,total_page+1):
                try:                                      
                    for element in response['results']:
                        self.images.append(element['urls']['regular'])   
                except Exception as e:
                    print(f"{i}번째 페이지에서 요청이 거부되었습니다. 한 시간 뒤에 {i}번째 페이지 부터 다시 시작 합니다!", e)
                    break
                self.start_page = i
                time.sleep(60*60+1)    
            
            print(f'추출한 image url의 개수는 {len(self.images)}입니다.')
        self.is_activate = True
    
    
    def execute_images(self):
        if self.is_activate and self.images:
            def save_images(image_url, paths, i):
                import base64
                            
                if 'data:' in str(image_url):
                    pass
                else:
                    try:
                        t= urlopen(image_url).read()
                        file = open(os.path.join(paths, str(i)+".gif"), 'wb')
                        file.write(t)
                    except Exception as e:
                        print(e)
            # 해당하는 폴더가 없을 경우 생성해주는 함수
            def makedirs(path): 
                try: 
                        os.makedirs(path) 
                except OSError: 
                    if not os.path.isdir(path): 
                        raise

            for i, image in enumerate(self.images,1):
                base_path = os.path.abspath(os.getcwd())
                save_path = base_path + f'/{self.query}'
                makedirs(save_path)
                
                save_images(image, save_path, i)
            print(f"저장한 경로 : {save_path}")
            return True
        else:
            if not self.images:
                print("크롤링한 image URL이 없습니다. 정상적으로 작동했는지 확인하세요.")
            elif not self.is_activate:
                print("크롤러가 활성화되어 있지 않습니다. activate 시켰는지 확인하세요.")

    def upload_db(self):
        import pymysql

        image_list =[[i,url] for i,url in enumerate(self.images,1)]
        con = pymysql.connect(host=self.env('DB_HOST'), port=3306, user=self.env('DB_USER'), passwd=self.env('DB_PASSWORD'), db=self.env('DB_NAME'), charset='utf8')
        cur = con.cursor()
        query = self.query
        try:
            print("이미 생성된 테이블이 있나 확인중 ...")
            CREATE_TABLE = f"""
                        CREATE TABLE {query} (
                            id INT NOT NULL,
                            url TEXT(255) NOT NULL,
                            PRIMARY KEY(id)
                        );            
            """
            cur.execute(CREATE_TABLE)
            con.commit()
        except Exception as e:
            print("이미 생성된 테이블이 존재합니다. 테이블을 지우고 다시 생성하여 진행합니다...")
            cur.execute(f"DROP TABLE {query}")
            cur.execute(CREATE_TABLE)
            con.commit()

        try:
            print("크롤링한 이미지 url을 데이터베이스에 적재합니다... ")
            cur.executemany(f"INSERT INTO {query} (id, url) VALUES (%s, %s)", image_list)
            con.commit()
        except Exception as e:
            print("데이터 적재에 실패했습니다. 오류코드 : ",e)
        con.close()

    def deactivate(self):
        self.is_activate = False
        return True
    
    def show(self):
        return self.images

In [84]:
c = crawler('asian')

In [85]:
c.activate()

total_pages = 307


In [86]:
c.show()

['https://images.unsplash.com/photo-1507591064344-4c6ce005b128?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=Mnw0MjE2ODV8MHwxfHNlYXJjaHwxfHxhc2lhbnxlbnwwfHx8fDE2Nzg3MzYzMDk&ixlib=rb-4.0.3&q=80&w=1080',
 'https://images.unsplash.com/photo-1544168190-79c17527004f?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=Mnw0MjE2ODV8MHwxfHNlYXJjaHwyfHxhc2lhbnxlbnwwfHx8fDE2Nzg3MzYzMDk&ixlib=rb-4.0.3&q=80&w=1080',
 'https://images.unsplash.com/photo-1487309078313-fad80c3ec1e5?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=Mnw0MjE2ODV8MHwxfHNlYXJjaHwzfHxhc2lhbnxlbnwwfHx8fDE2Nzg3MzYzMDk&ixlib=rb-4.0.3&q=80&w=1080',
 'https://images.unsplash.com/photo-1475823678248-624fc6f85785?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=Mnw0MjE2ODV8MHwxfHNlYXJjaHw0fHxhc2lhbnxlbnwwfHx8fDE2Nzg3MzYzMDk&ixlib=rb-4.0.3&q=80&w=1080',
 'https://images.unsplash.com/photo-1562904403-a5106bef8319?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=Mnw0MjE2ODV8MHwxfHNlYXJjaHw1fHxhc2lhbnxlbnwwfHx8fDE2Nzg3MzYzMDk&ixlib=rb-4.0.3&q=80&w=1080',
 

In [87]:
c.upload_db()

이미 생성된 테이블이 있나 확인중 ...
이미 생성된 테이블이 존재합니다. 테이블을 지우고 다시 생성하여 진행합니다...
크롤링한 이미지 url을 데이터베이스에 적재합니다... 
